In [1]:
import numpy as np
from chesslab_.utils import load_pkl
from chesslab_.training_tf import fitting
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [2]:
example=1
lr = 0.1
epochs=5
batch_size = 128
test_percent=0.1

path = 'D:/database/ccrl/'
name_data='ccrl_states_elo2.pkl'
name_labels='ccrl_results_elo2.pkl'
save_name='./tmp/tf_elo.0'

optim = tf.compat.v1.train.GradientDescentOptimizer(learning_rate=lr)
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [3]:
download=False
if download:
    from chesslab.utils import download_7z
    path='./'
    file_id = '1MFHFz_rxNziYSeN-9ruwnRiYskd0_9ss'
    download_7z(file_id,path)

In [4]:

encoding_1={
    '.':np.array([ 0, 0, 0],dtype=np.float),
    'p':np.array([ 0, 0, 1],dtype=np.float),
    'P':np.array([ 0, 0,-1],dtype=np.float),
    'b':np.array([ 0, 1, 0],dtype=np.float),
    'B':np.array([ 0,-1, 0],dtype=np.float),
    'n':np.array([ 1, 0, 0],dtype=np.float),
    'N':np.array([-1, 0, 0],dtype=np.float),
    'r':np.array([ 0, 1, 1],dtype=np.float),
    'R':np.array([ 0,-1,-1],dtype=np.float),
    'q':np.array([ 1, 0, 1],dtype=np.float),
    'Q':np.array([-1, 0,-1],dtype=np.float),
    'k':np.array([ 1, 1, 0],dtype=np.float),
    'K':np.array([-1,-1, 0],dtype=np.float)
}

encoding_2={
    '.':np.array([0,0,0,0],dtype=np.float),
    'p':np.array([1,0,0,0],dtype=np.float),
    'P':np.array([0,0,0,1],dtype=np.float),
    'b':np.array([0,1,0,0],dtype=np.float),
    'B':np.array([0,0,1,0],dtype=np.float),
    'n':np.array([1,1,0,0],dtype=np.float),
    'N':np.array([0,0,1,1],dtype=np.float),
    'r':np.array([1,0,1,0],dtype=np.float),
    'R':np.array([0,1,0,1],dtype=np.float),
    'q':np.array([1,0,0,1],dtype=np.float),
    'Q':np.array([0,1,1,0],dtype=np.float),
    'k':np.array([1,1,1,0],dtype=np.float),
    'K':np.array([0,1,1,1],dtype=np.float)
}


In [5]:
class Model_1():

    def __init__(self,
                   n_classes=2):
        initializer = tf.keras.initializers.GlorotNormal()
        self.hw=[]
        self.hb=[]

        self.hw.append( tf.Variable(initializer(shape=(7,7,3,32)),name="hl1weigths",dtype="float32") )
        self.hb.append( tf.Variable(np.zeros(32),name="hl1bias",dtype="float32") )
        #8x8x32
        self.hw.append( tf.Variable(initializer(shape=(5,5,32,64)),name="hl2weigths",dtype="float32"))
        self.hb.append( tf.Variable(np.zeros(64),name="hl2bias",dtype="float32"))
        #8x8x64
        self.hw.append( tf.Variable(initializer(shape=(3,3,64,128)),name="hl3weigths",dtype="float32"))
        self.hb.append( tf.Variable(np.zeros(128),name="hl3bias",dtype="float32"))
        #8x8x128
        self.hw.append( tf.Variable(initializer(shape=(8*8*128,256)),name="hl4weigths",dtype="float32"))
        self.hb.append( tf.Variable(np.zeros(256),name="hl4bias",dtype="float32"))

        self.hw.append( tf.Variable(initializer(shape=(256, n_classes)),name="outweigths",dtype="float32"))
        self.hb.append( tf.Variable(np.zeros(n_classes),name="outbias",dtype="float32"))

        self.trainable_variables = []
        for i in range(len(self.hw)):
            self.trainable_variables.append(self.hw[i])    
            self.trainable_variables.append(self.hb[i])

    def __call__(self,x): 

        # Declarando la arquitectura
        out = tf.cast(x, tf.float32)
        out = tf.reshape(out, shape=[-1, 8, 8, 3])

        layer=0
        out = tf.add(out, 1e-8)
        out = tf.nn.conv2d(out,self.hw[layer], strides=[1,1,1,1], padding='SAME')  
        out = tf.add(out, self.hb[layer])
        out = tf.nn.elu(out)
              #8*8*32
        layer+=1
        out = tf.nn.conv2d(out,self.hw[layer], strides=[1,1,1,1], padding='SAME')  
        out = tf.add(out, self.hb[layer])
        out = tf.nn.elu(out)
              #8*8*64
        layer+=1  
        out = tf.nn.conv2d(out,self.hw[layer], strides=[1,1,1,1], padding='SAME')  
        out = tf.add(out, self.hb[layer])
        out = tf.nn.elu(out)
              #8*8*128
        layer+=1
        out =  tf.reshape(out,[-1, 8*8*128])
        out =  tf.matmul(out,self.hw[layer])
        out = tf.add(out, self.hb[layer])
        out = tf.nn.elu(out)

        layer+=1
        out =  tf.matmul(out,self.hw[layer])
        out = tf.add(out, self.hb[layer])

        return out
    

class Model_2():

    def __init__(self,
                   n_classes=2):
        initializer = tf.keras.initializers.GlorotNormal()
        self.hw=[]
        self.hb=[]

        self.hw.append( tf.Variable(initializer(shape=(7,7,4,32)),name="hl1weigths",dtype="float32") )
        self.hb.append( tf.Variable(np.zeros(32),name="hl1bias",dtype="float32") )
        #8x8x32
        self.hw.append( tf.Variable(initializer(shape=(5,5,32,64)),name="hl2weigths",dtype="float32"))
        self.hb.append( tf.Variable(np.zeros(64),name="hl2bias",dtype="float32"))
        #8x8x64
        self.hw.append( tf.Variable(initializer(shape=(3,3,64,128)),name="hl3weigths",dtype="float32"))
        self.hb.append( tf.Variable(np.zeros(128),name="hl3bias",dtype="float32"))
        #8x8x128
        self.hw.append( tf.Variable(initializer(shape=(8*8*128,256)),name="hl4weigths",dtype="float32"))
        self.hb.append( tf.Variable(np.zeros(256),name="hl4bias",dtype="float32"))

        self.hw.append( tf.Variable(initializer(shape=(256, n_classes)),name="outweigths",dtype="float32"))
        self.hb.append( tf.Variable(np.zeros(n_classes),name="outbias",dtype="float32"))

        self.trainable_variables = []
        for i in range(len(self.hw)):
            self.trainable_variables.append(self.hw[i])    
            self.trainable_variables.append(self.hb[i])

    def __call__(self,x): 

        out = tf.cast(x, tf.float32)
        out = tf.reshape(out, shape=[-1, 8, 8, 4])

        layer=0
        out = tf.nn.conv2d(out,self.hw[layer], strides=[1,1,1,1], padding='SAME')  
        out = tf.add(out, self.hb[layer])
        out = tf.nn.relu(out)
              #8*8*32
        layer+=1
        out = tf.nn.conv2d(out,self.hw[layer], strides=[1,1,1,1], padding='SAME')  
        out = tf.add(out, self.hb[layer])
        out = tf.nn.relu(out)
              #8*8*64
        layer+=1  
        out = tf.nn.conv2d(out,self.hw[layer], strides=[1,1,1,1], padding='SAME')  
        out = tf.add(out, self.hb[layer])
        out = tf.nn.elu(out)
              #8*8*128
        layer+=1
        out =  tf.reshape(out,[-1, 8*8*128])
        out =  tf.matmul(out,self.hw[layer])
        out = tf.add(out, self.hb[layer])
        out = tf.nn.relu(out)

        layer+=1
        out =  tf.matmul(out,self.hw[layer])
        out = tf.add(out, self.hb[layer])

        return out

In [6]:
np.random.seed(0)
tf.random.set_seed(0)

x_data = load_pkl(path+name_data)[:100000]
y_data = load_pkl(path+name_labels)[:100000,1] #Nota: las etiquetas deben de ser enteros, no onehot

print(x_data.shape)
print(y_data.shape)

x_train, x_test, y_train, y_test = train_test_split(
    x_data, y_data, test_size = test_percent, random_state = 0, shuffle = True)

del x_data
del y_data

(100000, 64)
(100000,)


In [7]:
if example==1:
    model = Model_1()
    encoding=encoding_1
else:
    model = Model_2()
    encoding=encoding_2


In [8]:
fitting(epochs=epochs,
        x_train=x_train,
        y_train=y_train,
        x_test=x_test,
        y_test=y_test,
        model=model,
        optimizer=optim,
        batch_size=batch_size,
        lr=lr,
        loss_fn=loss_fn,
        save_name=save_name,
        encoding=encoding)

2021-10-24 13:45:18


Epoch: 01/05 | time: 8s = 0.1m | train loss: 0.6321 | train acc: 0.6424 | test loss: 0.6115 | test acc: 0.6677

Epoch: 02/05 | time: 5s = 0.1m | train loss: 0.6079 | train acc: 0.6611 | test loss: 0.5878 | test acc: 0.6790

Epoch: 03/05 | time: 5s = 0.1m | train loss: 0.5869 | train acc: 0.6787 | test loss: 0.5786 | test acc: 0.6874

Epoch: 04/05 | time: 5s = 0.1m | train loss: 0.5655 | train acc: 0.6956 | test loss: 0.5707 | test acc: 0.6996

Epoch: 05/05 | time: 5s = 0.1m | train loss: 0.5427 | train acc: 0.7128 | test loss: 0.5418 | test acc: 0.7221

In [9]:
fitting(epochs=1,
        x_train=x_train,
        y_train=y_train,
        x_test=x_test,
        y_test=y_test,
        model= model, 
        load_name='tmp/tf_elo.0.1.h5',
        save_name=save_name,)

2021-10-24 13:45:46


Epoch: 02/02 | time: 5s = 0.1m | train loss: 0.6069 | train acc: 0.6648 | test loss: 0.6041 | test acc: 0.6690